### Problem Statement
Traditional chatbots respond only to the current query and lack memory and access to
external knowledge. This limits their usefulness in real-world applications.

### Objective
The objective of this task is to build a context-aware chatbot using LangChain and
Retrieval-Augmented Generation (RAG) that:
- Remembers conversation history
- Retrieves relevant information from a document corpus
- Generates accurate, contextual responses


Install + Build Vector Store

In [1]:
!pip install -q langchain langchain-community langchain-openai faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# 🔐 Set OpenAI Key
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"

# Create sample corpus
with open("knowledge.txt", "w") as f:
    f.write("""
Artificial Intelligence is used in healthcare for diagnosis.
Machine learning helps predict diseases.
AI chatbots assist users with general questions.
""")

# Load & split documents
loader = TextLoader("knowledge.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# Create embeddings + vector store
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)


Context-Aware RAG Chatbot (Memory + Retrieval)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

# Memory for context
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# RAG chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory
)

# Chat function
def chat(query):
    return qa_chain({"question": query})["answer"]

# Test
chat("What is AI used for?")
chat("How does it help in healthcare?")


### Final Insights
- RAG improves accuracy by grounding responses in external knowledge.
- Context memory enables multi-turn conversations.
- FAISS provides fast and efficient document retrieval.
- LangChain simplifies orchestration of LLM, memory, and retriever.
